## NOTA: Porfavor correr el codigo ya que es muy pesado y el git hub no soporta el tamaño.

In [ ]:
#pip install plotly

In [ ]:
import pandas as pd
import os
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns


In [ ]:
file_name = 'Datos Contugas.xlsx'               
df_combined = pd.read_excel(file_name)
excel_data = pd.ExcelFile(file_name)


In [ ]:
df_combined.head()

In [ ]:
# Iterar sobre cada hoja y agregar el número de cliente ya que estan separados
for i, sheet_name in enumerate(excel_data.sheet_names, start=1):
    df_temp = excel_data.parse(sheet_name)
    df_temp['Numero_Cliente'] = f'CLIENTE{i}'
    df_combined = pd.concat([df_combined, df_temp], ignore_index=True)

In [ ]:
# Fecha' a datetime
df_combined['Fecha'] = pd.to_datetime(df_combined['Fecha'])
df_combined

In [ ]:
# Completitud
completitud_por_cliente = df_combined.groupby('Numero_Cliente').apply(lambda x: x.notnull().mean() * 100)

# Gráfico de barras para completitud por cliente
import plotly.express as px

# Crear gráfico de barras con plotly
fig = px.bar(completitud_por_cliente, 
             title="Completitud por Cliente", 
             labels={'value': 'Completitud (%)'})

# Ajustar tamaño de la figura
fig.update_layout(
    width=800,  # Ancho de la figura
    height=600  # Alto de la figura
)

# Mostrar el gráfico
fig.show()

In [ ]:
#Consistencia y claridad
# Obtener estadísticas descriptivas para todas las columnas numéricas
estadisticas_descriptivas = df_combined.describe()

# Convertir el DataFrame a formato tabular (texto plano)
tabla_texto = estadisticas_descriptivas.to_string()

# Imprimir la tabla para copiarla fácilmente
print(tabla_texto)

In [ ]:
##Formato
# Obtener los tipos de datos (formato) de cada columna
tipos_de_datos = df_combined.dtypes

# Convertir los tipos de datos a una tabla legible
tabla_formatos = tipos_de_datos.to_frame().reset_index()

# Renombrar las columnas para mayor claridad
tabla_formatos.columns = ['Variable', 'Formato']

# Convertir la tabla a un formato de texto tabular (para copiar y pegar)
tabla_formatos_texto = tabla_formatos.to_string(index=False)

# Imprimir la tabla para copiarla
print(tabla_formatos_texto)

In [ ]:
import plotly.express as px

# Crear un scatter plot de Temperatura contra Fecha
fig = px.scatter(df_combined, 
                 x='Fecha', 
                 y='Temperatura', 
                 title='Temperatura a lo largo del tiempo',
                 labels={'Fecha': 'Fecha', 'Temperatura': 'Temperatura (°C)'},
                 template='plotly_dark')

# Mostrar el gráfico
fig.show()

In [ ]:
import plotly.express as px

# Scatter plot para Volumen vs. Fecha (Tiempo)
fig_volumen = px.scatter(df_combined, 
                         x='Fecha', 
                         y='Volumen', 
                         title='Volumen a lo largo del tiempo',
                         labels={'Fecha': 'Fecha', 'Volumen': 'Volumen (m³)'},
                         template='plotly_dark') 

# Scatter plot para Presión vs. Fecha (Tiempo)
fig_presion = px.scatter(df_combined, 
                         x='Fecha', 
                         y='Presion', 
                         title='Presión a lo largo del tiempo',
                         labels={'Fecha': 'Fecha', 'Presion': 'Presión (Pa)'},
                         template='plotly_dark') 

# Mostrar ambos gráficos
fig_volumen.show()
fig_presion.show()

In [ ]:
# Crear una figura con tres subplots (uno para cada variable)
fig = make_subplots(rows=1, cols=3, subplot_titles=["Presion", "Temperatura", "Volumen"])

# Creamos un DF para saber qué clientes están disponibles
cliente_inicial = 'CLIENTE1'  # Solo seleccionar CLIENTE1

# Añadir los gráficos de caja para el cliente inicial
for i, variable in enumerate(['Presion', 'Temperatura', 'Volumen']):
    fig.add_trace(
        go.Box(y=df_combined[df_combined['Numero_Cliente'] == cliente_inicial][variable], 
               name=variable,
               boxpoints='all'),
        row=1, col=i+1
    )

# Actualizar el título del gráfico
fig.update_layout(
    title=f"Distribución de Presion, Temperatura y Volumen para {cliente_inicial}",
    yaxis_title="Valor",
)

# Mostrar el gráfico sin menú desplegable
fig.show()

In [ ]:
# Convertir la columna 'Fecha' a formato datetime
df_combined['Fecha'] = pd.to_datetime(df_combined['Fecha'])

# Obtener los primeros 5 clientes
clientes = df_combined['Numero_Cliente'].unique()[:2]

# Filtrar los datos para graficar por cada cliente
for cliente in clientes:
    df_cliente = df_combined[df_combined['Numero_Cliente'] == cliente]
    
    # Crear una figura para el cliente actual
    plt.figure(figsize=(12, 8))
    
    # Graficar cada variable
    plt.plot(df_cliente['Fecha'], df_cliente['Presion'], label='Presion')
    plt.plot(df_cliente['Fecha'], df_cliente['Temperatura'], label='Temperatura')
    plt.plot(df_cliente['Fecha'], df_cliente['Volumen'], label='Volumen')
    plt.xlabel('Fecha')
    plt.ylabel('Valor')
    plt.title(f'Presion, Temperatura y Volumen para {cliente}')
    plt.legend()
    plt.show()

In [ ]:
 #Valores nulos por columna
missing_data = df_combined.isnull().sum()
print("Valores nulos por columna:")
print(missing_data)

# Valores nulos por fila
rows_with_missing_data = df_combined.isnull().any(axis=1)
print("\nFilas con valores nulos:")
print(df_combined[rows_with_missing_data])

# Total de valores nulos en todo el DataFrame
total_missing_data = df_combined.isnull().sum().sum()
print(f'\nTotal de valores nulo base Contugas: {total_missing_data}')



In [ ]:
# Verificar si las columnas tienen el tipo de dato esperado
expected_types = {'Presion': float, 'Temperatura': float, 'Volumen': float}

for column, expected_type in expected_types.items():
    is_correct_type = df_combined[column].apply(lambda x: isinstance(x, expected_type)).all()
    if not is_correct_type:
        print(f"\nLa columna '{column}' tiene valores con tipo incorrecto.")

In [ ]:
# Función para detectar outliers usando el IQR
def detectar_outliers_iqr(df):
    columnas_numericas = df.select_dtypes(include=['float64', 'int64']).columns
    outliers = {}
    for columna in columnas_numericas:
        Q1 = df[columna].quantile(0.25)
        Q3 = df[columna].quantile(0.75)
        IQR = Q3 - Q1
        limite_inferior = Q1 - 1.5 * IQR
        limite_superior = Q3 + 1.5 * IQR
        outliers[columna] = df[(df[columna] < limite_inferior) | (df[columna] > limite_superior)]
    return outliers

# Función para detectar outliers usando la desviación estándar
def detectar_outliers_std(df):
    columnas_numericas = df.select_dtypes(include=['float64', 'int64']).columns
    outliers = {}
    for columna in columnas_numericas:
        media = df[columna].mean()
        desviacion_estandar = df[columna].std()
        limite_inferior = media - 3 * desviacion_estandar
        limite_superior = media + 3 * desviacion_estandar
        outliers[columna] = df[(df[columna] < limite_inferior) | (df[columna] > limite_superior)]
    return outliers

# Detectar outliers usando IQR
outliers_iqr = detectar_outliers_iqr(df_combined)

# Detectar outliers usando desviación estándar
outliers_std = detectar_outliers_std(df_combined)

# Mostrar los resultados
for columna, outlier_data in outliers_iqr.items():
    if not outlier_data.empty:
        print(f"\nOutliers en la columna '{columna}' (IQR):")
        print(outlier_data)

for columna, outlier_data in outliers_std.items():
    if not outlier_data.empty:
        print(f"\nOutliers en la columna '{columna}' (Desviación Estándar):")
        print(outlier_data)

# Visualizar los outliers con boxplots
plt.figure(figsize=(14, 8))
sns.boxplot(data=df_combined.select_dtypes(include=['float64', 'int64']))
plt.title("Boxplots de todas las columnas numéricas")
plt.show()


In [ ]:
# Calcular los cuartiles Q1 y Q3 de 'Volumen'
Q1 = df_combined['Volumen'].quantile(0.25)
Q3 = df_combined['Volumen'].quantile(0.75)

# Calcular el IQR (Rango Intercuartílico)
IQR = Q3 - Q1

# Definir los límites para los outliers
limite_inferior = Q1 - 1.5 * IQR
limite_superior = Q3 + 1.5 * IQR

# Identificar los outliers
outliers = df_combined[(df_combined['Volumen'] < limite_inferior) | (df_combined['Volumen'] > limite_superior)]

# Imputar aleatoriamente los outliers con valores entre Q1 y Q3
for index, row in outliers.iterrows():
    nuevo_valor = np.random.uniform(Q1, Q3)  
    df_combined.loc[index, 'Volumen'] = nuevo_valor  

# Verificación si quedan outliers después de la imputación
outliers_imputados = df_combined[(df_combined['Volumen'] < limite_inferior) | (df_combined['Volumen'] > limite_superior)]
print(f"Total de outliers restantes después de la imputación: {len(outliers_imputados)}")

# Boxplot después de imputar los outliers
plt.figure(figsize=(10, 6))
sns.boxplot(x=df_combined['Volumen'])
plt.title("Boxplot de 'Volumen' (Después de imputar outliers)")
plt.show()